# Lighthouse Macro — Live Data Notebook

*Sanitized version (EQL content removed).*

In [ ]:
# [REMOVED PROPRIETARY/EQL-SPECIFIC INTEGRATION]
# The original cell referenced EquiLend or private libs.
# It has been sanitized for Lighthouse Macro use.
# 
# # === 0) CONFIG ===
# import os
# 
# # --- API Keys ---
# # FRED (required for live macro): https://fred.stlouisfed.org/docs/api/api_key.html
# FRED_API_KEY = os.getenv("FRED_API_KEY", "7f8e44038ee69c4f78cf71873e85db16")
# 
# # Glassnode (optional): https://studio.glassnode.com/settings/api
# GLASSNODE_API_KEY = os.getenv("GLASSNODE_API_KEY", "")
# 
# # DefiLlama (public, but you can also cache locally if desired)
# # No key needed; we provide a fetch function you can disable if running offline.
# 
# # --- File Inputs (Lighthouse Macro, cached macro/crypto) ---
# DATA_DIR = os.getenv("LHM_DATA_DIR", "./data")
# EQL_CSV_BORROW = os.path.join(DATA_DIR, "equilend_borrow_timeseries.csv")  # columns: date,ticker,borrow_qty,utilization,fee_bps
# EQL_CSV_BUCKETS = os.path.join(DATA_DIR, "equilend_buckets_cc50.csv")      # columns: date,ticker,cc50,sscore,dl50
# CACHE_DIR = os.path.join(DATA_DIR, "cache")
# os.makedirs(DATA_DIR, exist_ok=True)
# os.makedirs(CACHE_DIR, exist_ok=True)
# 
# # --- Plotting ---
# SAVE_DIR = os.getenv("LHM_SAVE_DIR", "./outputs")
# os.makedirs(SAVE_DIR, exist_ok=True)


In [ ]:

# === 1) IMPORTS ===
import pandas as pd
import numpy as np
import requests
import io
import json
import warnings
from datetime import datetime, timedelta

# plotting
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# data
from fredapi import Fred
import yfinance as yf

warnings.filterwarnings("ignore")

# === 1a) Helpers ===
def to_dt(x):
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT


In [ ]:

# === 2) LIGHTHOUSE PALETTE & CHART HELPERS ===
PALETTE = {
    "blue": "#0052FF",
    "orange": "#FF8C00",
    "green": "#00A86B",
    "gray": "#6E7B8B",
    "purple": "#5D3FD3",
    # Optional alternates
    "magenta": "#C2185B",
    "light_blue": "#66B2FF"
}

def last_label(ax, x, y, color_key="blue", fmt="{:,.2f}"):
    if len(x) == 0 or len(y) == 0:
        return
    ax.annotate(fmt.format(y[-1]), xy=(x[-1], y[-1]), xytext=(6, 0),
                textcoords="offset points", color=PALETTE[color_key], fontsize=10,
                va="center")

def lh_style(ax, y_label="", color_key="blue", title=None, watermark=None):
    # Design rules
    ax.grid(False)
    # Right axis primary
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    # Spines
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_color("#A0A0A0")
    ax.spines["right"].set_color("#A0A0A0")
    # Labels/Title
    if y_label:
        ax.set_ylabel(y_label)
    if title:
        ax.set_title(title, loc="left", fontsize=12, color=PALETTE["gray"])
    # Watermark (optional path to PNG with transparency)
    if watermark and os.path.isfile(watermark):
        try:
            import matplotlib.offsetbox as offsetbox
            arr_img = plt.imread(watermark)
            imagebox = offsetbox.AnnotationBbox(offsetbox.OffsetImage(arr_img, zoom=0.15),
                                                (0.98, 0.12), frameon=False, xycoords='axes fraction')
            ax.add_artist(imagebox)
        except Exception:
            pass


### 3) FRED Pulls — Macro & Plumbing

In [ ]:

fred = Fred(api_key=FRED_API_KEY)

# Commonly used series (extend as needed)
FRED_SERIES = {
    # Rates & Curve
    "DGS10": "10Y Treasury Yield (%)",
    "DGS2": "2Y Treasury Yield (%)",
    "TB3MS": "3M T-Bill (%)",
    # Policy & Plumbing
    "RRPONTSYD": "ON RRP (Level, $ bn)",
    "SOFR": "SOFR (%)",
    "OBFR": "OBFR (%)",
    # Labor & Activity
    "JTSJOL": "Job Openings (JOLTS, millions)",
    "UNRATE": "Unemployment Rate (%)",
    # Inflation
    "PCEPI": "PCE Price Index",
    "PCEPILFE": "PCE less Food & Energy",
    # Credit & Financial Conditions (examples)
    "BAMLH0A0HYM2": "HY OAS (bps)",
    "BAMLC0A0CM": "IG OAS (bps)",
    # Sentiment
    "UMCSENT": "UMich Sentiment"
}

def fred_df(series_map, start="2000-01-01"):
    out = {}
    for s, name in series_map.items():
        try:
            ser = fred.get_series(s, observation_start=start)
            out[name] = ser
        except Exception as e:
            print(f"[FRED] {s} failed: {e}")
    df = pd.DataFrame(out)
    df.index.name = "date"
    return df

macro = fred_df(FRED_SERIES, start="2000-01-01")

# Derived plumbing metrics
if {"SOFR", "OBFR"}.issubset(set(FRED_SERIES.keys())):
    if "SOFR (%)" in macro.columns and "OBFR (%)" in macro.columns:
        macro["OBFR-SOFR (bp)"] = (macro["OBFR (%)"] - macro["SOFR (%)"]) * 100.0

# 3M Bill - SOFR
if "3M T-Bill (%)" in macro.columns and "SOFR (%)" in macro.columns:
    macro["TB3M - SOFR (bp)"] = (macro["3M T-Bill (%)"] - macro["SOFR (%)"]) * 100.0

macro = macro.sort_index()
macro.tail(3)


### 4) Cross-Asset Prices (yfinance placeholders)

In [ ]:

def yf_hist(ticker, start="2000-01-01"):
    try:
        df = yf.download(ticker, start=start, progress=False)
        df = df.rename(columns={"Adj Close":"adj_close"})
        df.index.name = "date"
        return df[["adj_close"]].dropna()
    except Exception as e:
        print(f"[YF] {ticker} failed: {e}")
        return pd.DataFrame()

assets = {
    "Gold": "GC=F",
    "BTC": "BTC-USD",
    "SPX": "^GSPC",
    "TLT": "TLT",
    "IEF": "IEF",
    "TMF": "TMF"
}

prices = {}
for name, tick in assets.items():
    prices[name] = yf_hist(tick, start="2005-01-01")

# Example merge for a chart
px = prices["Gold"].join(prices["BTC"], lsuffix="_gold", rsuffix="_btc", how="outer")
px.tail(3)


### 5) Stablecoins & Glassnode (optional)

In [ ]:

def fetch_defillama_stablecoins():
    url = "https://stablecoins.llama.fi/stablecoins?includePrices=false"
    try:
        r = requests.get(url, timeout=20)
        r.raise_for_status()
        data = r.json()
        # Total supply time series
        ts = data.get("peggedUSD", {}).get("totalCirculating", [])
        if not ts:
            return pd.DataFrame()
        df = pd.DataFrame(ts)
        df["date"] = pd.to_datetime(df["date"], unit="s")
        df = df.rename(columns={"totalCirculatingUSD":"supply_usd"})
        df = df.set_index("date").sort_index()
        return df[["supply_usd"]]
    except Exception as e:
        print(f"[DefiLlama] failed: {e}")
        return pd.DataFrame()

stable_supply = fetch_defillama_stablecoins()

# Glassnode placeholder (requires API key)
def fetch_glassnode_metric(metric="mvrv", asset="BTC"):
    if not GLASSNODE_API_KEY:
        print("[Glassnode] No API key set; skipping.")
        return pd.DataFrame()
    # Example endpoint placeholder; adapt to your endpoint/plan
    url = f"https://api.glassnode.com/v1/metrics/{metric}"
    params = {"a": asset, "api_key": GLASSNODE_API_KEY, "i":"24h"}
    try:
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        arr = r.json()
        df = pd.DataFrame(arr)
        df["t"] = pd.to_datetime(df["t"], unit="s")
        df = df.set_index("t").rename(columns={"v": metric}).sort_index()
        return df
    except Exception as e:
        print(f"[Glassnode] {metric} failed: {e}")
        return pd.DataFrame()

btc_mvrv = fetch_glassnode_metric(metric="mvrv", asset="BTC")


### 6) Lighthouse Macro CSV ingestion — Borrow/Utilization/Fee & Buckets

In [ ]:
# [REMOVED PROPRIETARY/EQL-SPECIFIC INTEGRATION]
# The original cell referenced EquiLend or private libs.
# It has been sanitized for Lighthouse Macro use.
# 
# def read_equilend_timeseries(path=EQL_CSV_BORROW):
#     if not os.path.isfile(path):
#         print(f"[Lighthouse Macro] Missing {path}. Provide a CSV with columns: date,ticker,borrow_qty,utilization,fee_bps")
#         return pd.DataFrame()
#     df = pd.read_csv(path)
#     df["date"] = pd.to_datetime(df["date"])
#     df = df.sort_values(["ticker","date"]).set_index("date")
#     return df
# 
# def read_equilend_buckets(path=EQL_CSV_BUCKETS):
#     if not os.path.isfile(path):
#         print(f"[Lighthouse Macro] Missing {path}. Provide a CSV with columns: date,ticker,cc50,sscore,dl50")
#         return pd.DataFrame()
#     df = pd.read_csv(path)
#     df["date"] = pd.to_datetime(df["date"])
#     df = df.sort_values(["ticker","date"]).set_index("date")
#     return df
# 
# eql_ts = read_equilend_timeseries()
# eql_bk = read_equilend_buckets()
# 
# # Example: Aggregate ETF lenses (TLT, IEF, TMF) and event target (e.g., FL/DKS)
# def eql_summary(tickers=("TLT","IEF","TMF"), start="2024-01-01"):
#     if eql_ts.empty:
#         return pd.DataFrame()
#     sub = eql_ts.loc[eql_ts.index >= pd.to_datetime(start)].copy()
#     sub = sub[sub["ticker"].isin(tickers)]
#     agg = sub.groupby(["date"]).agg(
#         borrow_qty=("borrow_qty","sum"),
#         utilization=("utilization","mean"),
#         fee_bps=("fee_bps","mean")
#     )
#     return agg
# 
# eql_bond_etf = eql_summary()
# eql_bond_etf.tail(3)


### 7) Signals — Plumbing Liquidity Index (PLI) & Credit Stress

In [ ]:

def zscore(s):
    s = s.astype(float)
    return (s - s.mean())/s.std(ddof=0)

def build_pli(df_macro):
    parts = []
    # Example components (choose available columns)
    if "ON RRP (Level, $ bn)" in df_macro.columns:
        parts.append(zscore(df_macro["ON RRP (Level, $ bn)"]))
    if "OBFR-SOFR (bp)" in df_macro.columns:
        parts.append(zscore(df_macro["OBFR-SOFR (bp)"]))
    if "TB3M - SOFR (bp)" in df_macro.columns:
        parts.append(zscore(df_macro["TB3M - SOFR (bp)"]))
    if len(parts)==0:
        return pd.Series(dtype=float)
    pli = pd.concat(parts, axis=1).mean(axis=1)
    return pli.rename("PLI (z)")

def build_credit_stress(df_macro):
    parts = []
    for col in ["HY OAS (bps)", "IG OAS (bps)"]:
        if col in df_macro.columns:
            parts.append(zscore(df_macro[col]))
    if len(parts)==0:
        return pd.Series(dtype=float)
    cs = pd.concat(parts, axis=1).mean(axis=1)
    return cs.rename("Credit Stress (z)")

pli = build_pli(macro)
credit_z = build_credit_stress(macro)

sig = pd.concat([pli, credit_z], axis=1).dropna(how="all")
sig.tail(3)


### 8) Chart Examples — Lighthouse Style

In [ ]:
# [REMOVED PROPRIETARY/EQL-SPECIFIC INTEGRATION]
# The original cell referenced EquiLend or private libs.
# It has been sanitized for Lighthouse Macro use.
# 
# def plot_series(df, cols, title, ylab="", color_keys=None, watermark=None, save_name=None):
#     fig, ax = plt.subplots(figsize=(9,4.5))
#     if color_keys is None:
#         color_keys = ["blue"]*len(cols)
#     for c, ck in zip(cols, color_keys):
#         s = df[c].dropna()
#         ax.plot(s.index, s.values, lw=2, color=PALETTE[ck], label=c)
#         last_label(ax, s.index.values, s.values, color_key=ck, fmt="{:,.2f}")
#     lh_style(ax, y_label=ylab, title=title, watermark=watermark)
#     ax.legend(frameon=False, loc="upper left")
#     plt.tight_layout()
#     if save_name:
#         path = f"{SAVE_DIR}/{save_name}"
#         plt.savefig(path, dpi=200, bbox_inches="tight")
#         print(f"Saved: {path}")
#     plt.show()
# 
# # 8a) Plumbing & Credit
# if not sig.empty:
#     plot_series(sig, ["PLI (z)", "Credit Stress (z)"],
#                 title="Plumbing Liquidity Index & Credit Stress (z-scores)",
#                 ylab="z-score", color_keys=["blue","orange"],
#                 save_name="lhm_pli_credit.png")
# 
# # 8b) Gold vs BTC (log, indexed)
# if not px.empty:
#     base_idx = px.dropna().iloc[0]
#     idx = px / base_idx
#     fig, ax = plt.subplots(figsize=(9,4.5))
#     for col, ck in zip(idx.columns, ["orange","purple"]):
#         s = idx[col].dropna()
#         ax.plot(s.index, np.log(s.values), lw=2, color=PALETTE[ck], label=col.replace("_"," ").title())
#         last_label(ax, s.index.values, np.log(s.values), color_key=ck, fmt="{:,.2f}")
#     lh_style(ax, y_label="log index", title="Gold vs BTC — Log Indexed Performance")
#     ax.legend(frameon=False, loc="upper left")
#     plt.tight_layout()
#     path = f"{SAVE_DIR}/lhm_gold_btc.png"
#     plt.savefig(path, dpi=200, bbox_inches="tight")
#     print(f"Saved: {path}")
#     plt.show()
# 
# # 8c) Lighthouse Macro Bond ETF Summary (if available)
# if eql_bond_etf is not None and not eql_bond_etf.empty:
#     plot_series(eql_bond_etf, ["borrow_qty","utilization","fee_bps"],
#                 title="Lighthouse Macro — Bond ETF Lending Summary",
#                 ylab="Level", color_keys=["blue","green","purple"],
#                 save_name="lhm_eql_bond_etf.png")
